In [1]:
import avro
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [2]:
import gdal
import numpy
zonefilename = "examples/data/states.shp"
datafilename = "examples/data/glc2000.tif"

In [3]:
schema = avro.schema.Parse(open("geotiff.avsc", "rb").read().decode('utf-8'))
writer = DataFileWriter(open("geotiff.avro", "wb"), DatumWriter(), schema, codec = 'deflate')

In [4]:
filehandle = gdal.Open(datafilename)
if filehandle is None:
    print ('Unable to open %s', datafilename)
    sys.exit(1)
Y = filehandle.RasterYSize
X = filehandle.RasterXSize
band = filehandle.GetRasterBand(1)
no_data_value = band.GetNoDataValue()
data_array = band.ReadAsArray(0,0,X,Y).flatten()

In [5]:
writer.append({"data": data_array.tobytes(), "nodatavalue": no_data_value, "X":X, "Y":Y})
writer.close()

In [6]:
import gc
del data_array
data_array = None
gc.collect()

181

In [ ]:
#reader = DataFileReader(open("geotiff.avro", "rb"), DatumReader())
#for data in reader:
#    print(data)
#reader.close()

In [7]:
import pyspark
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("Test")
sc = pyspark.SparkContext(conf = conf)

In [11]:
df = sc.read.format("com.databricks.spark.avro").load("geotiff.avro")

AttributeError: 'SparkContext' object has no attribute 'read'